In [1]:
import cv2 as cv  # Changed from 'import cv2'
import dlib
import numpy as np
import os

In [ ]:

# Load Dlib models (pre-trained)
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

# Authorized employees (known faces)
authorized_faces = {
    "aymene": None,
    "yacine": None,
    "moh": None,

}

def compute_face_encoding(image_path):
    if not os.path.exists(image_path):
        print(f"Error: Image file not found - {image_path}")
        return None
    img = cv.imread(image_path)
    if img is None:
        print(f"Error: Could not load image - {image_path}")
        return None
    rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    dets = detector(rgb_img, 1)
    if len(dets) == 0:
        print(f"No face found in {image_path}")
        return None
    shape = predictor(rgb_img, dets[0])
    encoding = np.array(facerec.compute_face_descriptor(rgb_img, shape))
    return encoding

# Pre-load authorized faces
authorized_faces["aymene"] = compute_face_encoding("authorized_faces/aymene.png")
authorized_faces["yacine"] = compute_face_encoding("authorized_faces/yacine.jpg")
authorized_faces["moh"] = compute_face_encoding("authorized_faces/moh.jpg")


if all(encoding is None for encoding in authorized_faces.values()):
    print("Error: No valid face encodings loaded. Check your reference images.")
    exit()

print("Loaded authorized employees:", [name for name, enc in authorized_faces.items() if enc is not None])

# Initialize webcam
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open laptop webcam.")
    exit()

cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

print("Camera ON - Running face verification prototype. Green for authorized, Red for unauthorized. Press 'q' to quit.")

# Night mode toggle (set True to test low-light adjustments)
night_mode = False # Change to True for nighttime simulation

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame from webcam.")
        break

    # Nighttime adjustments
    if night_mode:
        frame = cv.convertScaleAbs(frame, alpha=1.5, beta=50)  # Boost brightness/contrast
        rgb_frame = cv.cvtColor(cv.cvtColor(frame, cv.COLOR_BGR2GRAY), cv.COLOR_GRAY2RGB)  # Grayscale to RGB
    else:
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    dets = detector(rgb_frame, 1)

    for det in dets:
        shape = predictor(rgb_frame, det)
        face_encoding = np.array(facerec.compute_face_descriptor(rgb_frame, shape))

        name = "Unauthorized"
        is_authorized = False
        for auth_name, auth_encoding in authorized_faces.items():
            if auth_encoding is None:
                continue
            dist = np.linalg.norm(face_encoding - auth_encoding)
            if dist < 0.6:  # Security threshold
                name = auth_name
                is_authorized = True
                break

        # Draw rectangle and label with color coding
        left, top, right, bottom = det.left(), det.top(), det.right(), det.bottom()
        color = (0, 255, 0) if is_authorized else (0, 0, 255)  # Green for authorized, Red for unauthorized
        cv.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv.putText(frame, name, (left, top - 10), cv.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv.imshow("Security Camera Prototype - Face Verification", frame)

    # Exit on 'q'
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv.destroyAllWindows()

Loaded authorized employees: ['aymene', 'yacine', 'moh']
Camera ON - Running face verification prototype. Green for authorized, Red for unauthorized. Press 'q' to quit.
